In [4]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [21]:
#cache data for efficient performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('Visualization_Dashboard_Inventory_Data - Sheet1.csv')

    pn.state.cache['data'] = df.copy()

else:

    df = pn.state.cache['data']

In [6]:
#cache data for efficient performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('Visualization_Dashboard_Inventory_Data - Sheet1.csv')

    pn.state.cache['data'] = df.copy()

else:

    df = pn.state.cache['data']

In [7]:
#Read CSV file, handling potential BOM by using UTF-8-SIG encoding
df = pd.read_csv("Visualization_Dashboard_Inventory_Data - Sheet1.csv", encoding="utf-8-sig")

#Removes excess spaces, replace white spaces with underscores, and made all headers lowercase
cols = (df.columns
          .str.strip()
          .str.replace(r"\s+", "_", regex=True)
          .str.lower())

#Remove duplicates from remaining column name collisions and appendex (.1,.2,...) any duplicates
counts = {}
deduped = []
for c in cols:
    if c in counts:
        counts[c] += 1
        deduped.append(f"{c}_{counts[c]}")
    else:
        counts[c] = 0
        deduped.append(c)

df.columns = deduped

print(df.columns.tolist())
df

['product_id', 'name', 'category', 'units_in_stock', 'units_sold', 'price(usd)']


,product_id,name,category,units_in_stock,units_sold,price(usd)
0,100,CPU,Internal,100,40,250
1,101,RAM,Internal,250,100,70
2,102,Monitor,External,70,20,270
3,103,Keyboard,External,120,80,70
4,104,Mouse,External,180,70,15
5,105,PC Case,External,150,101,80


In [9]:
#Fill NAs with 0s and create price per unit sold column
df = df.fillna(0)
df['price_per_unit_sold'] = np.where(df['price(usd)']!= 0, df['price(usd)']/ df['units_sold'], 0)

In [10]:
#Makes pipelines interactive
idf = df.interactive()

In [13]:
product_id= ['100', '101', '102', '103', '104', '105']
price_per_unit_v_units_sold_pipeline = (
    df
    .groupby(['price(usd)', 'category', 'price_per_unit_sold'])['units_sold']
    .mean()
    .to_frame()
    .reset_index()
    .sort_values(by='category')
    .reset_index(drop=True)
)

In [15]:
price_per_unit_v_units_sold_pipeline

,price(usd),category,price_per_unit_sold,units_sold
0,15,External,0.214286,70.0
1,70,External,0.875000,80.0
2,80,External,0.792079,101.0
3,270,External,13.500000,20.0
4,70,Internal,0.700000,100.0
5,250,Internal,6.250000,40.0


In [17]:
#create price per units sold v units sold scatterplot using hvplot
price_per_unit_v_units_sold_scatterplot = price_per_unit_v_units_sold_pipeline.hvplot(x='price_per_unit_sold',
                                                                y='units_sold',
                                                                by='price(usd)',
                                                                size=80, kind="scatter",
                                                                alpha=0.7,
                                                                legend=False,
                                                                height=500,
                                                                width=500)
price_per_unit_v_units_sold_scatterplot

:NdOverlay   [price(usd)]
   :Scatter   [price_per_unit_sold]   (units_sold)

In [19]:
# Create a radio button widget for selecting the metric to visualize
y_picker = pn.widgets.RadioButtonGroup(
    name='Metric', options=['units_sold', 'units_in_stock', 'price_usd'],
    value='units_sold', button_type='success'
)
# Define a function to generate a bar chart of aggregated metric values by categor
@pn.depends(y_picker)
def bar_by_category(ycol):
    agg = (df.groupby('category', as_index=False)[ycol].sum()
             .sort_values(by=ycol, ascending=False))
    return agg.hvplot.bar(x='category', y=ycol, title=f'{ycol} by category')
# Create a Panel layout with metric selector
pn.Column(y_picker, bar_by_category)
# Create a radio button widget for selecting a category
cat_picker = pn.widgets.RadioButtonGroup(
    name='Category', options=sorted(df['category'].unique()), value='Internal',
    button_type='success'
)
# Create radio button widget for selecting a metric for the second plot
y_picker = pn.widgets.RadioButtonGroup(
    name='Metric', options=['units_sold', 'units_in_stock', 'price_usd'],
    value='units_sold', button_type='success'
)
# Generates bar chart with metric values
@pn.depends(cat_picker, y_picker)
def bar_products_for_category(cat, ycol):
    d = df[df['category'].eq(cat)]
    agg = (d.groupby('name', as_index=False)[ycol].sum()
             .sort_values(by=ycol, ascending=False))
    return agg.hvplot.bar(x='name', y=ycol,
                          title=f'{ycol} by product — {cat}', rot=45)

pn.Column(cat_picker, y_picker, bar_products_for_category)

Column
    [0] RadioButtonGroup(button_type='success', name='Category', options=['External', 'Internal'], value='Internal')
    [1] RadioButtonGroup(button_type='success', name='Metric', options=['units_sold', ...], value='units_sold')
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [20]:
#Layout using Template
template = pn.template.FastListTemplate(
    #Left sidebar layout and text
    title='Inventory Dashboard',
    sidebar=[pn.pane.Markdown("Inventory Visualization"),
             pn.pane.Markdown("#### Data visualization allows us to see datasets in a clear way in which we can understand and analyze. Charts, graphs, and interactive dashboards show significant trends and outliers in data that may not be as noticeable without visualization. In this case, the data for a hypothetical computer hardware store is shown."),
             pn.pane.JPG('inventory.jpg', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),
             ],
    main=[
        #Main body of layout and text which includes both graphs
        pn.Row(
            pn.Column(pn.pane.HoloViews(price_per_unit_v_units_sold_scatterplot, width=600),
                      margin=(0,25)),
            pn.Column(cat_picker, y_picker, bar_products_for_category)   # or bound_plot
        )
    ],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
#Makes the template servable
template.servable();